In [13]:
import pandas as pd

df = pd.read_csv("telco.csv")
print("File loaded! Shape:", df.shape)

df['Offer'] = df['Offer'].fillna('No Offer')
df['Internet Type'] = df['Internet Type'].fillna('No Internet')

df = df.drop(columns=[
    'Churn Score', 'CLTV', 'Churn Category', 'Churn Reason',
    'Country', 'State', 'Zip Code', 'Latitude', 'Longitude', 'Population'
])

df.to_csv("telco_clean.csv", index=False)
print("Done! Clean file saved.")

File loaded! Shape: (7043, 50)
Done! Clean file saved.


In [14]:
import os
print(os.getcwd())

/home/bcadf331-edb3-4804-b5bd-2c716cea11c5


In [15]:
import duckdb

conn = duckdb.connect()

# Example query — churn by contract type
result = conn.execute("""
    SELECT 
        Contract,
        COUNT(*) as Total_Customers,
        SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) as Churned,
        ROUND(SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as Churn_Rate_Pct
    FROM read_csv_auto('telco_clean.csv')
    GROUP BY Contract
    ORDER BY Churn_Rate_Pct DESC
""").df()

print(result)

# Save for Tableau
result.to_csv("tableau_churn_by_contract.csv", index=False)
```

Run a separate query for each insight you want in Tableau — churn by internet type, churn by payment method, churn by tenure etc. Save each one as its own CSV.

---

**STEP 3 — Connect to Tableau**

Two options:

**Option A — Simplest:** Just drag your CSV files straight into Tableau Desktop. Go to Connect → Text File → select your CSV. Done.

**Option B — Use the full clean file:** Drag `telco_clean.csv` into Tableau and build all your charts from there directly. Tableau can do its own filtering and grouping so you don't even need the DuckDB query files.

---

**My recommendation for you:** Use Option B — drag `telco_clean.csv` directly into Tableau and build everything there. DuckDB is great for practice and shows on your portfolio that you know SQL, but Tableau is powerful enough to handle the aggregations itself on a dataset this size.

---

**So your full flow is:**
```
telco.csv → Python cleaning → telco_clean.csv → Tableau
                ↓
           DuckDB queries (optional but good for portfolio)

SyntaxError: invalid character '—' (U+2014) (3594720370.py, line 23)

In [20]:
import os
print(os.getcwd())

/home/bcadf331-edb3-4804-b5bd-2c716cea11c5


In [1]:
import pandas as pd

df = pd.read_csv("telco_clean.csv")

# 1. Churn by Contract
contract = df.groupby("Contract").apply(
    lambda x: pd.Series({
        "Total_Customers": len(x),
        "Churned": (x["Churn Label"] == "Yes").sum(),
        "Churn_Rate_Pct": round((x["Churn Label"] == "Yes").sum() * 100 / len(x), 2)
    })
).reset_index()
contract.to_csv("churn_by_contract.csv", index=False)
print("Contract done!")

# 2. Churn by Internet Type
internet = df.groupby("Internet Type").apply(
    lambda x: pd.Series({
        "Total_Customers": len(x),
        "Churned": (x["Churn Label"] == "Yes").sum(),
        "Churn_Rate_Pct": round((x["Churn Label"] == "Yes").sum() * 100 / len(x), 2)
    })
).reset_index()
internet.to_csv("churn_by_internet.csv", index=False)
print("Internet done!")

# 3. Churn by Payment Method
payment = df.groupby("Payment Method").apply(
    lambda x: pd.Series({
        "Total_Customers": len(x),
        "Churned": (x["Churn Label"] == "Yes").sum(),
        "Churn_Rate_Pct": round((x["Churn Label"] == "Yes").sum() * 100 / len(x), 2)
    })
).reset_index()
payment.to_csv("churn_by_payment.csv", index=False)
print("Payment done!")

# 4. Churn by Tenure
tenure = df.groupby("Tenure in Months").apply(
    lambda x: pd.Series({
        "Total_Customers": len(x),
        "Churned": (x["Churn Label"] == "Yes").sum(),
        "Churn_Rate_Pct": round((x["Churn Label"] == "Yes").sum() * 100 / len(x), 2)
    })
).reset_index()
tenure.to_csv("churn_by_tenure.csv", index=False)
print("Tenure done!")

print("All done! Ready for Tableau!")

Contract done!
Internet done!
Payment done!
Tenure done!
All done! Ready for Tableau!


/tmp/ipykernel_697/3919433799.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  contract = df.groupby("Contract").apply(
/tmp/ipykernel_697/3919433799.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  internet = df.groupby("Internet Type").apply(
/tmp/ipykernel_697/3919433799.py:28: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pan